# Parser 9.00 - Scalar Numeric Variables

The goal of this version of the parser is for it to recognize symbol names that stand for numeric values.

Actually the symbol names will not be true variables because there will not yet be any way to assign a value to them.

>If you object that “variables” which cannot be altered are hardly what most programmers think of as variables – well, you’re right. Picky, picky, picky…

Despite this limitation there are still some things to think about. One decision we have to make is whether or not to require variables to be declared before we can use them (as in [C](https://en.wikipedia.org/wiki/C_(programming_language)) and [Pascal](https://en.wikipedia.org/wiki/Pascal_(programming_language)) )) or give them a default value if they’ve never been seen before first encountering them in an expression (as in [BASIC](https://en.wikipedia.org/wiki/BASIC) and [AWK](https://en.wikipedia.org/wiki/AWK)).

Requiring variables to be declared before use enables better error checking but adds overhead and slows down a parser. This is not usually a problem for compiled languages like C and Pascal. They will do the error checking and pay the time price just once at compile time. At execution time there is no cost for these, because they’ve already been done.

Languages like BASIC and AWK, which are often implemented as interpreted, have no “once and done” compile stage. If they did not provide default values they would have to pay the price of error checking over and over again each time the same piece of code was executed. It is also easier to write parsers and evaluators that provide default values.

As our parser and evaluator resemble an interpreter more than a compiler, we will adopt the default value approach. The parser will be able to treat symbol names as (almost) just another type of operand to be placed into the Reverse Polish. The evaluator will handle all the work of managing them (which in this version will not amount to much, as all symbol names will stand for the same value).

>If you object that there also [just-in-time compilers](https://en.wikipedia.org/wiki/Just-in-time_compilation) and [bytecode interpreters](https://en.wikipedia.org/wiki/Bytecode) that blur the distinction between compilers and interpreters...well, you're right. So noted.

## Libraries

In [ ]:
import glob       # for searching directories
import math       # for 'log--()' functions

import re         # for regular expressions

## User output

In [ ]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [ ]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, stk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', stk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Parser

In [ ]:
# operands accepted:
# - decimal and hexadecimal floating point literals
# - scalar numeric variables

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '9.00'
    
    _FLTMAX =  4294967295                                  # 2**32-1
    _FLTMIN = -4294967296                                  # -(2**32)

    _expMax = {
        'P' : math.log2(_FLTMAX),                          # max base 2 exponent
        'E' : math.log10(_FLTMAX)                          # max base 10 exponent
    }

    _unPrefxOp = '[-+!]'                                   # unary operators
 
    _unPrec = {                                            # unary operator precedence
        '*': 100,
        '-': 80, '+': 80, '!': 80
    }
    
    _binInfxOp = '[-+*/]|==|!='                            # binary operators
        
    _binPrec = {                                           # binary operator precedence
        '*': 70, '/': 70,
        '-': 60, '+': 60,
        '==': 50, '!=': 50
    }
    

    def __init__(self):
        pass
           
    def doparse(self, this):

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                 # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        stk = [ ('EOE', 1) ]       # operator stack

        def parseErr(mesg):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            UIwriteln(f'{"^^near here".rjust(start)}')
            return False

        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            while prec <= stk[-1][1]:
                result.append(stk.pop()[0])

        def pushLeft(op, prec):
            '''push left associative operator on stack'''
            popGEop(prec)
            stk.append( (op, prec) )

        def popGop(prec):
            '''pop operators of greater precedence'''
            while prec < stk[-1][1]:
                result.append(stk.pop()[0])

        def pushRight(op, prec):
            '''push right associative operator on stack'''
            popGop(prec)
            stk.append( (op, prec) )

        def popUntil(op, prec):
            '''clear and check operator stack'''
            popGEop(prec)
            if op == stk.pop()[0]:      # top remaining operator is the one we want to see ?
                return True
            elif op == '(':
                return parseErr('Unmatched right parenthesis')
            elif op == 'EOE':
                return parseErr('Unmatched left parenthesis')
       
        def convertFloat(fplit, base, capgrp):
            '''convert floating point literal to internal form'''
            
            def rangeErr():
                return parseErr(f'\'{fplit}\' is out of range')
            
            # collect the features of interest
                   
            p = re.search(capgrp, fplit.upper())
                            
            lint, lfrc, expbas, expsgn, lexp = p.group(1,2,4,5,6)
            
            # convert integer portion (if any)
 
            uint = 0
            if lint:
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return rangeErr()
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc:
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return rangeErr()
            
            # value so far
            
            uflt = uint + ufrc
 
            # convert exponent portion (if any)
            
            uexp = 0
            if lexp:
                for ch in lexp:
                    digval = '0123456789'.find(ch)
                    if uexp <= (self._expMax[expbas] - digval)/10:
                        uexp =  uexp * 10 + digval
                    else:
                        return rangeErr()
                    
            # adjust value by exponent (if any)
             
            if uexp:
                power = (2 if expbas == 'P' else 10) ** uexp
                if expsgn == '-':
                    uflt /= power
                elif uflt <= self._FLTMAX/power:
                    uflt *= power
                else:
                    return rangeErr()
                    
            result.append(uflt)
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop

        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    stk.append( ('(', 2) )


                elif startsWith(self._unPrefxOp):
                    '''unary prefix ?'''
                    pushRight( 'U' + token, self._unPrec[token] )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)([pP][-+]?[0-9]+)?'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16, '0X([0-9A-F]*)[.]?([0-9A-F]*)(([P])([-+])?([0-9]+))?')

                    elif startsWith('([0-9]+([.][0-9]*)?|[.][0-9]+)([eE][-+]?[0-9]+)?'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10, '([0-9]*)[.]?([0-9]*)(([E])([-+])?([0-9]+))?')
                        
                    elif startsWith('[a-zA-Z][_a-zA-Z0-9]*'):
                        '''numeric scalar variable?'''
                        result.append( token.upper() )
                        pushLeft( 'U*', self._unPrec['*'] )
                        
                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand')

            # look for operator

            else:

                if startsWith('[)]'):
                    ok = popUntil( '(', 4 )

                else:

                    wantoperand = True                                # flip

                    if startsWith(self._binInfxOp):
                        '''binary infix ?'''
                        pushLeft( 'B' + token, self._binPrec[token] )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator')

            PTshowparse(ok, result, stk )                         # trace

        if ok:
            if wantoperand:
                ok = parseErr('Unexpected end of expression')     # must be in 'wantoperator' state   
            else:
                ok = popUntil( 'EOE', 3 )                         # clear operator stack

        return (ok, result if ok else None)                       # done

### How it works

First we must decide what an acceptable variable name looks like:

```Python
'[a-zA-Z][_a-zA-Z0-9]*'
```

which basically means "an alphabetic character, optionally followed by zero or more underscores and alphanumeric characters".

>This definition is slightly different from what many languages (C and [Python](https://en.wikipedia.org/wiki/Python_(programming_language)) among them) consider legal names:
```Python
'[_a-zA-Z][_a-zA-Z0-9]*'
```
>which means ""an underscore or alphabetic character, optionally followed by zero or more underscores and alphanumeric characters". This small distinction  means that a single underscore character is a legal identifier. So are two underscores, or three, or however many in a row are permitted before a compiler chokes on them. The fact that human readers find it nearly impossible to understand code that employs such identifiers is one of the features of C that makes the [International Obfuscated C Code Contest](https://en.wikipedia.org/wiki/International_Obfuscated_C_Code_Contest) possible.

Note that a symbol name cannot be confused with a numeric literal because a symbol name cannot start with a digit character and a numeric literal must (a subtle use of *manifest typing*). Symbol names used as variables are also different from numeric literals in that whatever value they stand for cannot be determined by the parser. Any range checking will have to be done by the evaluator.

In the absence of variable declaration and the error detection possibilities that affords, the most a parser can do is “add a note” in the Reverse Polish that means “look this value up” at evaluation time. Here the “note” will take the form of an operator which does not appear in the original infix, *U\**. The particular symbol used mimics C's [pointer dereference operator](https://en.wikipedia.org/wiki/Dereference_operator). When executed by the evaluator, it replaces the symbol name on the top of the operand stack with the value it represents.

When a variable name is found, the parser immediately adds its uppercase version to the Reverse Polish (because case sensitivity is [evil](https://www.tonymarston.net/php-mysql/case-sensitive-software-is-evil.html), [evil](https://blog.codinghorror.com/the-case-for-case-insensitivity/), [evil](https://tiamat.tsotech.com/case-sensitivity-sucks) ), then pushes *U\** onto the operator stack. Its high precedence means that the next operator pushed onto the stack will pop it into the Reverse Polish immediately after the variable name. Which is what we want.

>If you object that it would be more straightforward to simply add the *U\** operator directly to the Reverse Polish just as we did the variable name it applies to...well, you're right. That would also alleviate any need to give *U\** any precedence at all. But this way emphasizes that despite orginating as something added by the parser, *U\** is otherwise the same as any operator appearing in the original infix and can be treated same way. Also, *U\** is just the first of several operators the parser will add to the Reverse Polish.

# Evaluator

In [ ]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality
# - variable name de-reference

# errors detected:
# - out of range
# - division by zero

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        pass
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True
        
        def unVal(var):
            '''variable name value'''
            return pushOperand(0)

#        def unPlu(arg):
#           '''unary plus'''
#           return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )

        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            
            # negative minus negative
            # required: lft - rgt <= FLTMAX
            
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )

        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            if lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft*rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft*rgt )

            else:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMAX / lft, lft * rgt )

        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return inRange( True, lft/rgt )
            
            # rgt positive ?
            
            elif rgt > 0:
                
                # lft positive ?
                # req: lft/rgt <= FLTMAX
                
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                
                # req: lft/rgt >= FLTMIN
                
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
                
            # rgt negative ?
            
            elif rgt < 0:
                
                # lft positive ?
                # req: lft/rgt >= FLTMIN
                
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )       # reverse inequality
                 
                # req: lft/rgt <= FLTMAX
                
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )        # reverse inequality
                
            # rgt is zero

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        # initialize
                                
        unDispatch = {
            'U*': unVal,
            'U-': unNeg,
            'U+': pushOperand,
            'U!': unNot
        }
        
        binDispatch = {
            'B+': binAdd,
            'B-': binSub,
            'B*': binMul,
            'B/': binDiv,
            'B==': binEqu,
            'B!=': binNeq
        }
  
        stk = []
        ok = True

        # main loop
        
        for v in rpn:

            ETshowtoken(v)
            
            if v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

*U\** is a unary operator like any other. It has a function to perform its operation and an entry in the dispatch table pointing to that function.

All that happens in this version is to push the default value represented by all variable names onto the operand stack.

>That value can be anything we like, but it is generally best to choose a useful one. Most often that turns out to be zero.

The argument passed to the function is the name of the variable to de-reference. Since every one produces the same default value they don’t matter to the result, and so we can ignore it. In the next version we'll change that.

## Running the parser

In [ ]:
passCnt = failCnt = 0                       # most useful for test input files, but never any harm

myParser = myEvaluator = None               # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser

def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [ ]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests